In [10]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '/p/project/chhb19/gaertner2/lkf_detection/functions/')
from statistics_functions import *
sys.path.insert(1, '/p/project/chhb19/gaertner2/lkf_tools/lkf_tools/')
from dataset import *
import numpy as np
import pandas as pd

In [2]:
path = '/p/project/chhb19/gaertner2/data/awicm_cvmix/'
path_ds = path + 'datasets/'

In [3]:
years_all = np.load(path + 'years_all.npy')
a_mean, a_tot, h_mean, h_tot = np.load(path + 'a_mean_tot_h_mean_tot.npy')

In [4]:
years = [2013, 2017, 2020, 2093, 2097, 2100]

In [5]:
#years = [i for i in range(2093,2097)]

inds = [np.where(years_all==year)[0][0] for year in years]
a_tot_sel = a_tot[inds]
files = [path_ds + f'ds_{year}.npy' for year in years]

In [6]:
# load the data from the lkf_data files
datasets, lkfs = get_lkfs(files)

res_km = get_res_km(datasets[0])

In [7]:
###### Calculate Metrics ######

tracks = get_tracks(datasets)
# paths_all also includes the paths going through each timestep
paths, paths_all = get_paths(lkfs, tracks)


n_lkfs = get_n_lkfs(lkfs)
rho_lkfs = n_lkfs / a_tot_sel * 10000
length, av_length, total_length = get_lkf_length(lkfs, res_km)
lifetimes, mean_lifetime = get_lkf_lifetimes(paths)
# lifetimes_all includes the lifetimes of LKFs that are
# already counted in previous timesteps
lifetimes_all, _ = get_lkf_lifetimes(paths_all)

np.save(path + 'data_lkfs_paths.npy', np.array([years, lkfs, paths, paths_all], dtype='object'))

/tmp/ipykernel_23687/1523436045.py:9: RuntimeWarning: invalid value encountered in divide
  rho_lkfs = n_lkfs / a_tot_sel * 10000
/p/project/chhb19/gaertner2/jupyter/kernels/gaertner2_lkf_tools/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/p/project/chhb19/gaertner2/jupyter/kernels/gaertner2_lkf_tools/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/p/project/chhb19/gaertner2/lkf_detection/functions/statistics_functions.py:210: RuntimeWarning: Mean of empty slice.
  mean_lifetime_y += lifetimes_timestep.mean(),
/p/project/chhb19/gaertner2/jupyter/kernels/gaertner2_lkf_tools/lib/python3.10/site-packages/numpy/core/_methods.py:131: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret / rcount


In [8]:
years

[2013, 2017, 2020, 2093, 2097, 2100]

In [9]:
###### Create LKF Dictionary ######

# i is the first index of years in the 2090s, used in the averaging function av_sd
i = 3
LKFs = dict()

for decade, j in zip(['2010s', '2090s'], [[0,1], [2,3]]):

    df = pd.DataFrame(av_sd(n_lkfs,i)[j[0]], columns=['number'])
    df['number sd']        = av_sd(n_lkfs,i)[j[1]]
    df['density']          = av_sd(rho_lkfs,i)[j[0]]
    df['density sd']       = av_sd(rho_lkfs,i)[j[1]]
    df['mean length']      = av_sd(av_length,i)[j[0]]
    df['mean length sd']   = av_sd(av_length,i)[j[1]]
    df['total length']     = av_sd(total_length,i)[j[0]]
    df['total length sd']  = av_sd(total_length,i)[j[1]]
    df['mean lifetime']    = av_sd(mean_lifetime,i)[j[0]]
    df['mean lifetime sd'] = av_sd(mean_lifetime,i)[j[1]]

    LKFs[decade] = df

for y in range(len(years)):
    df_y = pd.DataFrame(n_lkfs[y], columns=['number'])
    df_y['density']       = rho_lkfs[y]
    df_y['mean length']   = av_length[y]
    df_y['total length']  = total_length[y]
    df_y['mean lifetime'] = mean_lifetime[y]
    
    LKFs[f'y{years[y]}'] = dict(
        total = df_y,
    )
    
    for d in range(365):
        df_d = pd.DataFrame(length[y][d], columns=['length'])
        df_d['lifetime'] = lifetimes_all[y][d]
        
        LKFs[f'y{years[y]}'][f'd{d}'] = df_d
        
np.save(f'{path}LKFs', [LKFs])